빌보드 hot100 크롤러

1. 몇 건 수집할 지 입력받기 
2. 파일 저장 경로 입력받기 
3. 시작 타임스탬프 찍기 
4. 빌보드 hot100 페이지 접속
5. For 반복문으로 page_down 반복해서 페이지 맨 아래까지 내리기 
6. 페이지 파싱
7. 파싱해서 페이지 항목들로 구성된 리스트 만들기 
8. For 반복문으로 리스트 돌면서 항목 하나하나 정보 수집하기 
9. No +=1 (기본값 = 1)
10. If no > cnt : break

f.close()
driver.quit()

1. 데이터프레임 생성
2. 데이터프레임 항목별로 칼럼에 추가하기 
3. 데이터프레임 csv, xlsx로 저장하기 
4. 데이터프레임 불러와서 엑셀 이미지파일 저장하기 
5. 종료 타임스탬프 찍기
6. 전체 총 소요시간 계산하기 
7. 사용자에게 총 요청건수, 실제 수집 건수 보여주기 
8. 사용자에게 총 소요시간 계산한거 보여주기 
종료 

In [1]:
import selenium
from selenium import webdriver
import os
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from PIL import Image
from io import BytesIO
import requests

In [2]:
print('-'*100)
print('Billboard Hot 100 차트 크롤러')
print('+'*100)

try : 
    cnt = int(input('1.빌보드 핫100 차트는 총 100개 곡이 랭크되어 있습니다. 몇 번까지 수집할까요? : '))
except : 
    cnt = 100
f_dir = input('2.파일 저장 경로를 입력하세요 : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+'빌보드 핫100')
os.chdir(f_dir+s+'-'+'빌보드 핫100')

f_name = s+'-'+'빌보드 핫100'+'.txt'
fc_name = s+'-'+'빌보드 핫100'+'.csv'
fx_name = s+'-'+'빌보드 핫100'+'.xlsx'

----------------------------------------------------------------------------------------------------
Billboard Hot 100 차트 크롤러
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
1.빌보드 핫100 차트는 총 100개 곡이 랭크되어 있습니다. 몇 번까지 수집할까요? : 100
2.파일 저장 경로를 입력하세요 : 


In [3]:
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
driver = webdriver.Chrome(path, options=options)

s_time = time.time()
driver.get('https://www.billboard.com/charts/hot-100')
time.sleep(5)

In [4]:
for a in range(25) : 
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(3) #페이지 컨텐츠 로드위해 페이지 맨 아래까지 내리기 
    
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
contents_list = soup.find('div', 'chart-list container ').find_all('li', 'chart-list__element display--flex')

In [5]:
no = 1
f = open(f_name, 'a', encoding='utf-8')
img_list1 = []
rank_list1 = []
trend_list1 = []
song_list1 = []
artist_list1 = []
last_list1 = []
peak_list1 = []
week_list1 = []

for b in contents_list : 
    print('-'*100)
    f.write('-'*100+'\n')
    
    img = b.find('span', 'chart-element__image flex--no-shrink')['style']
    img_src = img.replace('background-image: url("', "").replace('");', "").strip()
    img_list1.append(img_src)
    
    rank_number = b.find('span', 'chart-element__rank__number').get_text().replace('\n', "").strip()
    rank_list1.append(rank_number)
    print('1.순위 : %s' %rank_number)
    f.write('1.순위 : %s' %rank_number+'\n')
    
    trend = b.find('span', 'chart-element__rank flex--column flex--xy-center flex--no-shrink').find_all('span')[1].get_text().replace('\n', "").strip()
    trend_list1.append(trend)
    print('2.트렌드 : %s' %trend)
    f.write('2.트렌드 : %s' %trend+'\n')
    
    song_name = b.find('span','chart-element__information__song text--truncate color--primary').get_text().replace('\n', "").strip()
    song_list1.append(song_name)
    print('3.곡 제목 : %s' %song_name)
    f.write('3.곡 제목 : %s' %song_name+'\n')
    
    artist = b.find('span', 'chart-element__information__artist text--truncate color--secondary').get_text().replace('\n', "").strip()
    artist_list1.append(artist)
    print('4.아티스트 : %s' %artist)
    f.write('4.아티스트 : %s' %artist+'\n')
    
    try : 
        last = b.find('span', 'chart-element__meta text--center color--secondary text--last').get_text().replace('\n', "").strip()
        last_list1.append(last)
        print('5.last : %s' %last)
        f.write('5.last : %s' %last+'\n')
    except : 
        last_list1.append(' ')
        print('5.last : 제공된 정보 없음')
        f.write('5.last : 제공된 정보 없음'+'\n')
    
    try : 
        peak = b.find('span', 'chart-element__meta text--center color--secondary text--peak').get_text().replace('\n', "").strip()
        peak_list1.append(peak)
        print('6.peak : %s' %peak)
        f.write('6.peak : %s' %peak+'\n')
    except : 
        peak_list1.append(' ')
        print('6.peak : 제공된 정보 없음')
        f.write('6.peak : 제공된 정보 없음'+'\n')
        
    try : 
        week = b.find('span', 'chart-element__meta text--center color--secondary text--week').get_text().replace('\n', "").strip()
        week_list1.append(week)
        print('7.week : %s' %week)
        f.write('7.week : %s' %week+'\n')
    except : 
        week_list1.append(' ')
        print('7.week : 제공된 정보 없음')
        f.write('7.week : 제공된 정보 없음'+'\n')
        
    f.write('-'*100+'\n')
    print('-'*100)
    
        
        
    no += 1
    if no > cnt : 
        break
f.close()
driver.quit()


billboard = pd.DataFrame()
billboard['순위'] = rank_list1
billboard['트렌드'] = trend_list1
billboard['곡 제목'] = song_list1
billboard['아티스트'] = artist_list1
billboard['last'] = last_list1
billboard['peak'] = peak_list1
billboard['week'] = week_list1
billboard.to_csv(fc_name, encoding='utf-8-sig', index=False)
billboard.to_excel(fx_name, index=False)

#엑셀에 사진 넣기 
def pic_to_excel() : 
    wb = openpyxl.load_workbook(fx_name)
    ws = wb.worksheets[0]

    for a in range(2, no+1) : 
        ws.row_dimensions[a].height =  160
    
    ws.column_dimensions['C'].width = 50


    nor = 2
    for a in img_list1 : 
        res = requests.get(a)
        image_data = BytesIO(res.content)
        img = openpyxl.drawing.image.Image(image_data)
        ws.add_image(img, anchor='C%s' %nor)
        nor +=1
    wb.save(fx_name)
    print('엑셀에 사진 첨부 완료') 

pic_to_excel()

e_time = time.time()
total_time = e_time - s_time
print('='*100)
print('요청하신 %s건 중 %s건 만큼 데이터 수집했습니다' %(cnt, (no-1)))
print('데이터 수집하는 데 총 걸린 시간은 %s초 입니다' %round(total_time, 2))
print('txt파일 저장 경로 : %s' %f_dir+s+'-'+'빌보드 핫100'+'/'+f_name)
print('csv파일 저장 경로 : %s' %f_dir+s+'-'+'빌보드 핫100'+'/'+fc_name)
print('xlsx파일 저장 경로 : %s' %f_dir+s+'-'+'빌보드 핫100'+'/'+fx_name)
print('='*100)

----------------------------------------------------------------------------------------------------
1.순위 : 1
2.트렌드 : Steady
3.곡 제목 : Drivers License
4.아티스트 : Olivia Rodrigo
5.last : 1 LW
6.peak : 1 Pk
7.week : 6 WoC
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.순위 : 2
2.트렌드 : Rising
3.곡 제목 : 34+35
4.아티스트 : Ariana Grande
5.last : 6 LW
6.peak : 2 Pk
7.week : 16 WoC
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.순위 : 3
2.트렌드 : New
3.곡 제목 : Calling My Phone
4.아티스트 : Lil Tjay Featuring 6LACK
5.last : 
6.peak : 3 Pk
7.week : 1 WoC
----------------------------------------------------------------------------------------------------
----------------------------------------------------

엑셀에 사진 첨부 완료
요청하신 100건 중 100건 만큼 데이터 수집했습니다
데이터 수집하는 데 총 걸린 시간은 101.89초 입니다
txt파일 저장 경로 : /Users/kibeomkim/Desktop/웹크롤링_결과_데이터/2021-02-25-12-10-38-빌보드 핫100/2021-02-25-12-10-38-빌보드 핫100.txt
csv파일 저장 경로 : /Users/kibeomkim/Desktop/웹크롤링_결과_데이터/2021-02-25-12-10-38-빌보드 핫100/2021-02-25-12-10-38-빌보드 핫100.csv
xlsx파일 저장 경로 : /Users/kibeomkim/Desktop/웹크롤링_결과_데이터/2021-02-25-12-10-38-빌보드 핫100/2021-02-25-12-10-38-빌보드 핫100.xlsx
